In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf
from PIL import Image

from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import normalize, to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense 

In [3]:
image_directory='data/'
no_tumor_images=os.listdir(image_directory+ 'not_detected/')
yes_tumor_images=os.listdir(image_directory+ 'tumor_detected/')

print('No Tumor: ', len(no_tumor_images))
print('Tumor: ',len(yes_tumor_images))

No Tumor:  1500
Tumor:  1500


In [4]:
dataset=[]
label=[]

INPUT_SIZE=64


In [5]:
for i , image_name in enumerate(no_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'not_detected/'+image_name)
        image=Image.fromarray(image,'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(0)

In [6]:
for i , image_name in enumerate(yes_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'tumor_detected/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE))
        dataset.append(np.array(image))
        label.append(1)

In [9]:
ataset=np.array(dataset)
label=np.array(label)

print('Dataset: ',len(dataset))
print('Label: ',len(label))

Dataset:  3000
Label:  3000


In [10]:
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size=0.2, random_state=2023)

In [11]:
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)

In [12]:
model=Sequential()

model.add(Conv2D(32, (3,3),activation='relu', input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3),activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3),activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

In [13]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, 
batch_size=32, 
verbose=1, epochs=14, 
validation_data=(X_test, y_test),
shuffle=True)

Epoch 1/14
75/75 [==============================] - 11s 16ms/step - loss: 0.5651 - accuracy: 0.7133 - val_loss: 0.5073 - val_accuracy: 0.7733
Epoch 2/14
75/75 [==============================] - 1s 10ms/step - loss: 0.4344 - accuracy: 0.8046 - val_loss: 0.3630 - val_accuracy: 0.8483
Epoch 3/14
75/75 [==============================] - 1s 10ms/step - loss: 0.3429 - accuracy: 0.8529 - val_loss: 0.2861 - val_accuracy: 0.8850
Epoch 4/14
75/75 [==============================] - 1s 11ms/step - loss: 0.2674 - accuracy: 0.8958 - val_loss: 0.2465 - val_accuracy: 0.9033
Epoch 5/14
75/75 [==============================] - 1s 10ms/step - loss: 0.1934 - accuracy: 0.9262 - val_loss: 0.2216 - val_accuracy: 0.9050
Epoch 6/14
75/75 [==============================] - 1s 9ms/step - loss: 0.1377 - accuracy: 0.9533 - val_loss: 0.1498 - val_accuracy: 0.9400
Epoch 7/14
75/75 [==============================] - 1s 9ms/step - loss: 0.1067 - accuracy: 0.9629 - val_loss: 0.1567 - val_accuracy: 0.9500
Epoch 8/14
75/

In [15]:
model.save('BrainTumorDetection.h5')